In [1]:
import numpy as np
import tensorflow as tf
import keras
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras import applications
import os
import shutil
import random
%matplotlib inline

In [2]:
a = plt.imread("data/train/Closed/_3.jpg").shape
print(a)
cv2.resize(cv2.imread(os.path.join('data/train/Closed', '_3.jpg'), cv2.IMREAD_COLOR),(256,256))
a = plt.imread("data/train/Closed/_3.jpg").shape
print(a)

(224, 224, 3)
(224, 224, 3)


In [2]:
path_train = 'data/train'
path_test = 'data/test'
path_valid = 'data/valid'

In [3]:
train_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=path_train,target_size= (224,224), batch_size=10)
valid_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=path_valid,target_size= (224,224), batch_size=10)
test_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=path_test,target_size= (224,224), batch_size=10, shuffle=False)

Found 1012 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 240 images belonging to 2 classes.


In [8]:
model = load_model('model_mobile.h5')

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pw_1 (Conv2D)           (None, 112, 112, 64)     

In [7]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_generator,240/10)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Closed', 'Open']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[ 96  24]
 [  9 111]]
Classification Report
              precision    recall  f1-score   support

      Closed       0.91      0.80      0.85       120
        Open       0.82      0.93      0.87       120

    accuracy                           0.86       240
   macro avg       0.87      0.86      0.86       240
weighted avg       0.87      0.86      0.86       240



In [4]:
model_cnn = load_model('models/model_cnn.h5')

In [6]:
model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 200704)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 401410    
Total params: 420,802
Trainable params: 420,802
Non-trainable params: 0
________________________________________________

In [7]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model_cnn.predict(test_generator,240/10)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Closed', 'Open']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[117   3]
 [  3 117]]
Classification Report
              precision    recall  f1-score   support

      Closed       0.97      0.97      0.97       120
        Open       0.97      0.97      0.97       120

    accuracy                           0.97       240
   macro avg       0.97      0.97      0.97       240
weighted avg       0.97      0.97      0.97       240



### Due to large size of model_vgg.h5,unable to predict its accuracy on test_generator and I have prefered simple cnn model because of its higher accuracy